In [ ]:
from __future__ import print_function

import datetime
import numpy as np
import glob
import warnings

from multiprocessing import Pool

## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        
from clustered_SNe import data_dir_default
from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs, get_full_id_from_partial_id
    
from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status



In [ ]:
data_dir = data_dir_default

In [ ]:
def parse_into_simulations(data_dir_and_id):
    # wrapper function to allow multiprocessing + map
    data_dir, id = data_dir_and_id
    run_summary = RunSummary(data_dir=data_dir, id=id)

    simulation = Simulation.from_run_summary(run_summary)
    
    inputs = Inputs(os.path.join(data_dir, id + "_inputs.dat"))
    simulation_inputs = Simulation_Inputs.from_Inputs(id, inputs)
    
    simulation_status = Simulation_Status.from_run_summary(run_summary)
    
    
    return simulation, simulation_inputs, simulation_status
        

In [1]:
def map_multiple_ids(mappable_inputs, n_cpus=4):
    pool = Pool(n_cpus)
    simulation_tuples = pool.map(parse_into_simulations, mappable_inputs)
    return simulation_tuples

# Add particular ids

In [ ]:
ids = [
"adea4ef1-5b5f-4e12-bffd-977a8581946c",
]

In [ ]:
data_dir_tmp = "../tmp/"
data_dir_input = data_dir_tmp

In [ ]:
mappable_inputs = [(data_dir_input, id) for id in ids]

simulation_tuples = map_multiple_ids(mappable_inputs)

In [ ]:
for simulation, simulation_inputs, simulation_status in simulation_tuples:
        print(simulation_status.id + ": " + simulation_status.status)

In [ ]:
for simulation, simulation_inputs, simulation_status in simulation_tuples:
    if simulation_status.status in ["Error", "Unphysical", "Unknown"]:
        print(simulation.id, ": ", simulation_status.status )

In [ ]:
for simulation, simulation_inputs, simulation_status in simulation_tuples:
    simulation.add_or_update_to_table()
    simulation_inputs.add_or_update_to_table()
    simulation_status.add_or_update_to_table()

In [ ]:
session.commit()

# Add entire directory to database in batch

In [ ]:
def add_ids_to_database(data_dir, ids, n_cpus=4):

    mappable_inputs = [(data_dir, id) for id in ids]
    simulation_tuples = map_multiple_ids(mappable_inputs, n_cpus=n_cpus)

    for simulation, simulation_inputs, simulation_status in simulation_tuples:
        simulation.add_or_update_to_table()
        simulation_inputs.add_or_update_to_table()
        simulation_status.add_or_update_to_table()


In [ ]:
def add_entire_dir_to_database(data_dir, n_cpus=4):
    overview_filenames = glob.glob(os.path.join(data_dir, "*overview.dat"))
    ids = [None]*len(overview_filenames)
    for k, filename in enumerate(overview_filenames):
        ids[k] = os.path.basename(filename).split("_overview.dat")[0]

    add_ids_to_database(data_dir, ids, n_cpus=n_cpus)

# Check statuses from database

In [ ]:
statuses = np.array([status.status for status in session.query(Simulation_Status)])
counts = {status : 0 for status in Simulation_Status.possible_statuses}

for status in statuses:
    counts[status] += 1


print(counts)

In [ ]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Ready"):
    print(simulation_status.id)

In [ ]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Error"):
    print(simulation_status.id)

In [ ]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Running"):
    print(simulation_status.id)

In [ ]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Unphysical"):
    print(simulation_status.id)

In [ ]:
for simulation_status in session.query(Simulation_Status).filter(Simulation_Status.status == "Unknown"):
    print(simulation_status.id)

# Shut down SQL session when done

In [ ]:
session.commit()

In [ ]:
session.close()